In [1]:
import argparse
import json
import logging
import os
import random
import time

import torch
import numpy as np

In [2]:
import sys 
sys.path.append('..')
import utils.misc as ws
import utils.data_utils
import utils.train_utils
import utils.eval_utils
import utils.mesh
import utils.dataset as d
import models.networks as arch


In [3]:
DATA_SOURCE = '/home/ninad/Desktop/Docs/phd-res/proj-irvl-grasp-transfer/code/docker-data/output_dataset/'
# EXPERIMENTS_DIR = '../experiments/epochs_2000_weight_0.5/'
EXPERIMENTS_DIR = '../experiments/all3_toy_grp-pc-encoder/'
CHECKPOINT = 'latest'
split_filename = os.path.join(EXPERIMENTS_DIR, 'split_train.json')
                              
specs_filename = os.path.join(EXPERIMENTS_DIR, "specs.json")

LATENT_CODE_DIR = ws.latent_codes_subdir

In [4]:
specs = json.load(open(specs_filename))

latent_size = specs["CodeLength"]
gripper_weight = specs["GripperWeight"]
num_samp_per_scene = specs["SamplesPerScene"]

In [8]:
encoder_grp = arch.ResnetPointnet(c_dim=latent_size, dim=3, hidden_dim=256)

decoder = arch.dsdfDecoder(2*latent_size, **specs["NetworkSpecs"])

encoderDecoder = arch.MultiGripperDecoder(encoder_grp, decoder, num_samp_per_scene)

encoderDecoder = torch.nn.DataParallel(encoderDecoder)

saved_model_state = torch.load(
    os.path.join(
        EXPERIMENTS_DIR, ws.model_params_subdir, CHECKPOINT + ".pth")
)

saved_model_epoch = saved_model_state["epoch"]

encoderDecoder.load_state_dict(saved_model_state["model_state_dict"])

encoderDecoder = encoderDecoder.module.cuda()

In [9]:
with open(split_filename, "r") as f:
    split = json.load(f)

# npz_filenames = utils.data_utils.dsdf_get_instance_filenames(
#     args.data_source, split)
gpc_files, npz_filenames = utils.data_utils.get_instance_filelist(DATA_SOURCE, split)

# random.shuffle(npz_filenames) # WHY??? DISABLE THIS FOR CHECKING REPRODUCIBILITY

In [10]:
for f in npz_filenames:
    print(f[-35:])

df/fetch_gripper/sdf_graspnum_0.npz
000/sdf/Barrett/sdf_graspnum_10.npz
0/sdf/HumanHand/sdf_graspnum_20.npz


In [11]:
for gpc in gpc_files:
    print(gpc[-20:])

ud/fetch_gripper.npz
nt_cloud/Barrett.npz
_cloud/HumanHand.npz


In [12]:
latent_vecs = ws.load_latent_vectors(EXPERIMENTS_DIR, CHECKPOINT)
print(latent_vecs.shape)

torch.Size([3, 256])


In [13]:
index_to_select = random.randint(0, len(npz_filenames)-1)

npz = npz_filenames[index_to_select]
gpc = gpc_files[index_to_select]

full_filename = npz

print(index_to_select, npz[-35:])


1 000/sdf/Barrett/sdf_graspnum_10.npz


In [14]:
latent_vec = latent_vecs[index_to_select]

In [15]:
latent_vec = latent_vec.squeeze()
print(latent_vec.shape)

torch.Size([256])


In [17]:
# IMPORTANT: HAVE SHUFFLE = FALSE SO THAT SAME DATA POINT IS LOADED AS index_to_select

sdf_dataset = d.MultiGripperSamples(DATA_SOURCE, split, 1000000)

sdf_loader = torch.utils.data.DataLoader(
    sdf_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=8,
    drop_last=True )

In [18]:
# samples, npzfile = next(iter(sdf_loader))
gripper_pc, samples, idx, npzfile = sdf_dataset[index_to_select]
print(npzfile[-40:])

ale_1000/sdf/Barrett/sdf_graspnum_10.npz


In [16]:
####### MESH RECONSTRUCTION CODE!

is_gripper = True

if is_gripper:
    mesh_filename = os.path.join(EXPERIMENTS_DIR, f'test_{npz[-15:]}')
else:
    mesh_filename = os.path.join(EXPERIMENTS_DIR, f'test_o_{npz[-15:]}')


latent_vec = latent_vec.squeeze().cuda()
with torch.no_grad():
    utils.mesh.create_mesh_custom(
        encoderDecoder, latent_vec.cuda(), mesh_filename, gripper_pc=gripper_pc.cuda(), N=256, max_batch=int(2 ** 18), isGripper=is_gripper)


IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [19]:
queries = samples[:, :3] # Need to pass this through the network
gt_sdf_obj = samples[:, 3].squeeze().numpy()
gt_sdf_grp = samples[:, 4].squeeze().numpy()
print(gt_sdf_grp.shape)

(1000000,)


In [20]:
print(samples.shape)

torch.Size([1000000, 5])


In [21]:
print(gripper_pc.shape)

torch.Size([512, 3])


In [22]:
gripper_pc.unsqueeze(0).shape

torch.Size([1, 512, 3])

In [23]:
with torch.no_grad():
    queries, sdf_obj, sdf_grp = utils.eval_utils.eval_query_pc_multi(encoderDecoder, latent_vec.cuda(), queries.cuda(), gripper_pc.unsqueeze(0).cuda(), max_batch_size=16384)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16384 but got size 576 for tensor number 1 in the list.

In [ ]:
queries = queries.detach().cpu().numpy()
sdf_obj = sdf_obj.detach().cpu().numpy()
sdf_grp = sdf_grp.detach().cpu().numpy()

In [ ]:
print(sdf_grp.shape)
print(sdf_obj.shape)
print(queries.shape)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

def plot_sdf(xyz, sdf, title='Sample_Title', n_display=10000):
    fig = plt.figure(figsize = (10,10))
    ax = fig.add_subplot(111, projection='3d')    

    ind = np.random.choice(range(xyz.shape[0]), n_display)
    data = xyz[ind].T

    ax.scatter(data[0], data[2], data[1], s=5, c=sdf[ind])
    ax.view_init(20, 100)
    limit = (-0.95, 0.95)
    ax.set_xlim3d(*limit)
    ax.set_ylim3d(*limit)
    ax.set_zlim3d(*limit)
    plt.title(title)
    plt.show()

In [ ]:
EPS = -1e-4
# ind_obj =  np.where((y >= -EPS) & (y <= EPS))
ind_grp = sdf_grp <= EPS

plot_sdf(queries[ind_grp], sdf_obj[ind_grp])

min(sdf_grp)

In [ ]:
EPS = -1e-4
# ind_obj =  np.where((y >= -EPS) & (y <= EPS))
ind_grp = gt_sdf_grp <= EPS

plot_sdf(queries[ind_grp], gt_sdf_grp[ind_grp])

min(gt_sdf_obj)

In [ ]:
import pyrender
# Just pass the points you want to visualize
def plt_points_3d(pts):
    colors = np.zeros(pts.shape)
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

# Pass the point as well as the sdf to see inside/outside points
def plt_points_sdf(pts, sdf, eps=1e-4):
    colors = np.zeros(pts.shape)
    colors[sdf < eps, 1] = 1
    colors[sdf > eps, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)

def plt_points_sdf_compare(pts, sdf_gt, sdf_pred):
    colors = np.zeros(pts.shape)
    colors[sdf_gt < 0, 1] = 1
    colors[sdf_gt > 0, 0] = 1
    cloud = pyrender.Mesh.from_points(pts, colors=colors)
    scene = pyrender.Scene()
    scene.add(cloud)
    viewer = pyrender.Viewer(scene, use_raymond_lighting=True, point_size=2)
    
    colors2 = np.zeros(pts.shape)
    colors2[sdf_pred < 0, 1] = 1
    colors2[sdf_pred > 0, 0] = 1
    cloud2 = pyrender.Mesh.from_points(pts, colors=colors2)
    scene2 = pyrender.Scene()
    scene2.add(cloud2)
    viewer = pyrender.Viewer(scene2, use_raymond_lighting=True, point_size=2)

In [ ]:
plt_points_3d(queries[sdf_grp < 1e-4])

In [ ]:
plt_points_3d(queries[sdf_obj < 1e-1])

In [ ]:
plt_points_sdf(queries, sdf_grp)

In [ ]:
plt_points_sdf(queries, sdf_obj)